## Pclass, Gender, sibsp, parch를 독립변수로 사용하여 로지스틱 회귀모형을 실시하였을 때, parch변수의 계수값은? 단, Pclass는 범주형 변수이다
(반올림하여 소수 셋째 자리까지 계산)

In [1]:
import pandas as pd
df = pd.read_csv("/kaggle/input/bigdatacertificationkr/Titanic.csv")

In [2]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Gender       891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
from scipy.stats import chi2_contingency
# print(dir(scipy.stats))
# print(help(chi2_contingency))
table = pd.crosstab(df.Gender, df.Survived)
# table.head()
chi2, p_val, dof, exp = chi2_contingency(table)
print(round(chi2, 3)) # 260.717
print(p_val) # 1.1973570627755645e-58 
# 0.05 유의수준보다 현저히 작음 0에 가까움 -> 귀무가설 기각, Gender와 Survived는 서로 연관이 있다.

from statsmodels.formula.api import logit
result1 = logit("Survived ~ Gender + SibSp + Parch + Fare", data=df).fit().summary()
print(result1) # -0.201

result2 = logit("Survived ~ Gender + SibSp + Parch + Fare", data=df).fit().params

import numpy as np
print(round(np.exp(result2), 3)) # 0.702

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


260.717
1.1973570627755645e-58
Optimization terminated successfully.
         Current function value: 0.482065
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      886
Method:                           MLE   Df Model:                            4
Date:                Fri, 20 Jun 2025   Pseudo R-squ.:                  0.2761
Time:                        18:32:27   Log-Likelihood:                -429.52
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 1.192e-69
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.9466      0.169      5.590      0.000       0.61

In [4]:
from statsmodels.formula.api import logit
# print(dir(statsmodels.formula.api))
print(help(logit))

Help on method from_formula in module statsmodels.base.model:

from_formula(formula, data, subset=None, drop_cols=None, *args, **kwargs) method of builtins.type instance
    Create a Model from a formula and dataframe.
    
    Parameters
    ----------
    formula : str or generic Formula object
        The formula specifying the model.
    data : array_like
        The data for the model. See Notes.
    subset : array_like
        An array-like object of booleans, integers, or index values that
        indicate the subset of df to use in the model. Assumes df is a
        `pandas.DataFrame`.
    drop_cols : array_like
        Columns to drop from the design matrix.  Cannot be used to
        drop terms involving categoricals.
    *args
        Additional positional argument that are passed to the model.
    **kwargs
        These are passed to the model with one exception. The
        ``eval_env`` keyword is passed to patsy. It can be either a
        :class:`patsy:patsy.EvalEnvironm

In [5]:
formula = "Survived ~ C(Pclass) + Gender + SibSp + Parch"
# 예측하려는 종속 변수: Survived
# C(Pclass)라고 쓰면 범주형(categorical) 변수로 처리
# Pclass Gender + SibSp + Parch는 설명 변수(독립 변수)
model = logit(formula, data = df).fit()
model.params # -0.049847 반올림 -0.050

Optimization terminated successfully.
         Current function value: 0.459565
         Iterations 6


Intercept         2.491729
C(Pclass)[T.2]   -0.848152
C(Pclass)[T.3]   -1.866905
Gender[T.male]   -2.760281
SibSp            -0.232553
Parch            -0.049847
dtype: float64

## 로지스틱 회귀 계수 해석
로지스틱 회귀의 계수는 로그 오즈(log-odds) 단위

### Parch의 계수 -0.050의 의미

- Parch가 1 증가할 때마다 생존할 로그 오즈가 0.050만큼 감소

- 즉, 동반한 자녀나 부모가 많을수록 생존 가능성이 약간 줄어든다는 뜻

## 풀이

In [6]:
import pandas as pd
from statsmodels.formula.api import logit

df = pd.read_csv("/kaggle/input/bigdatacertificationkr/Titanic.csv")
formula = "Survived ~ C(Pclass) + Gender + SibSp + Parch"
model = logit(formula, data=df).fit()
model.params

Optimization terminated successfully.
         Current function value: 0.459565
         Iterations 6


Intercept         2.491729
C(Pclass)[T.2]   -0.848152
C(Pclass)[T.3]   -1.866905
Gender[T.male]   -2.760281
SibSp            -0.232553
Parch            -0.049847
dtype: float64